In [ ]:
%pip install lark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 2.1 MB/s eta 0:00:00


In [ ]:
from lark import Lark, Transformer

# Define the grammar
grammar = """
    grammar_start: paragraph
    paragraph: sentence+
    sentence: country " is" " a" " country" " with" " capital " capital ". " additional_info
    country: COUNTRY
    capital: CAPITAL
    additional_info: ADDITIONAL_INFO
    COUNTRY: "India" | "United States" | "United Kingdom" | "France" | "Germany" | "Japan" | "Brazil" | "Canada" | "Australia" | "China"
    CAPITAL: "New Delhi" | "Washington, D.C" | "London" | "Paris" | "Berlin" | "Tokyo" | "Brasília" | "Ottawa" | "Canberra" | "Beijing"
    ADDITIONAL_INFO: "It is known for its rich culture and history. " | "It is a major economic powerhouse. " | "It is famous for its landmarks and attractions. " | "It has a diverse population. " | "It is renowned for its cuisine."
"""


class CountryCapitalTransformer(Transformer):
    def grammar_start(self, items):
        country_info = {}
        print(items[0])
        for paragraph in items:
            for sentence in paragraph:
                country, capital, info = sentence
                country_info[country] = {'capital': capital, 'additional_info': info}
        return country_info

    def paragraph(self, items):
        return items

    def sentence(self, items):
        return (items[0], items[1], items[2])

    def country(self, items):
        return str(items[0])

    def capital(self, items):
        return str(items[0])

    def additional_info(self, items):
        return str(items[0])

# Define the parser
parser = Lark(grammar, start='grammar_start', parser='lalr', transformer=CountryCapitalTransformer())

# Example paragraph
paragraph_text = "India is a country with capital New Delhi. It is known for its rich culture and history. United States is a country with capital Washington, D.C. It is a major economic powerhouse. France is a country with capital Paris. It is famous for its landmarks and attractions. Germany is a country with capital Berlin. It has a diverse population. Japan is a country with capital Tokyo. It is renowned for its cuisine."

# Parse the paragraph
countries_info = parser.parse(paragraph_text)

# Print the syntax tree
print(countries_info)


[('India', 'New Delhi', 'It is known for its rich culture and history. '), ('United States', 'Washington, D.C', 'It is a major economic powerhouse. '), ('France', 'Paris', 'It is famous for its landmarks and attractions. '), ('Germany', 'Berlin', 'It has a diverse population. '), ('Japan', 'Tokyo', 'It is renowned for its cuisine.')]
{'India': {'capital': 'New Delhi', 'additional_info': 'It is known for its rich culture and history. '}, 'United States': {'capital': 'Washington, D.C', 'additional_info': 'It is a major economic powerhouse. '}, 'France': {'capital': 'Paris', 'additional_info': 'It is famous for its landmarks and attractions. '}, 'Germany': {'capital': 'Berlin', 'additional_info': 'It has a diverse population. '}, 'Japan': {'capital': 'Tokyo', 'additional_info': 'It is renowned for its cuisine.'}}


In [ ]:
print(countries_info['India'])

{'capital': 'New Delhi', 'additional_info': 'It is known for its rich culture and history. '}


In [ ]:
from lark import Lark, Transformer

# Define the grammar
grammar = """
    grammar_start: question
    question: capital_question | additional_info_question
    capital_question: "What is the capital of " country "?"
    additional_info_question: "What is the additional info about " country "?"
    country: COUNTRY
    COUNTRY: "India" | "United States" | "United Kingdom" | "France" | "Germany" | "Japan" | "Brazil" | "Canada" | "Australia" | "China"
"""

# Custom transformer class
class CountryInfoTransformer(Transformer):
    def __init__(self, country_info):
        self.country_info = country_info

    def question(self, items):
        return items[0]

    def capital_question(self, items):
        country = items[0]
        if country in self.country_info:
            return f"The capital of {country} is {self.country_info[country]['capital']}."
        else:
            return f"Sorry, I don't have information about {country}."

    def additional_info_question(self, items):
        country = items[0]
        if country in self.country_info:
            return f"The additional info about {country} is: {self.country_info[country]['additional_info']}"
        else:
            return f"Sorry, I don't have information about {country}."

    def country(self, items):
        return str(items[0])

# Define the parser
parser = Lark(grammar, start='grammar_start', parser='lalr')

# Custom transformer instance
transformer = CountryInfoTransformer(countries_info)

# Example questions
questions = [
    "What is the capital of India?",
    "What is the additional info about Japan?",
    "What is the capital of Brazil?",
    "What is the additional info about United Kingdom?"
]

# Answer the questions
for question in questions:
    parsed_question = parser.parse(question)
    answer = transformer.transform(parsed_question)
    print(answer.children[0])

The capital of India is New Delhi.
The additional info about Japan is: It is renowned for its cuisine.
Sorry, I don't have information about Brazil.
Sorry, I don't have information about United Kingdom.
